In [35]:
!pip install yfinance
%pip install openai==1.12.0



StatementMeta(, 8f0536b4-a6c3-4390-b53d-22459a26b32d, 48, Finished, Available)


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [36]:
from notebookutils.mssparkutils.credentials import getSecret

keyvault = "https://finanaceaiKeyvault.vault.azure.net/"

openaiapi = getSecret(keyvault, "openai-api")

StatementMeta(, 8f0536b4-a6c3-4390-b53d-22459a26b32d, 50, Finished, Available)

In [37]:
import yfinance as yf
import pandas as pd
from datetime import datetime

# List of ticker symbols
ticker_symbols = 'MSFT AAPL 2222.SR GOOGL AMZN NVDA META BRK-A LLY TSM PEP SAP TMO LIN 601857.SS ROG.SW NVS MCD ABT DIS'

# Fetch data for all tickers
tickers = yf.Tickers(ticker_symbols)

# Prepare a list to collect all news items
all_news = []

# Iterate through each ticker to fetch its news
for ticker_symbol in ticker_symbols.split():
    # Access the specific Ticker object
    ticker = tickers.tickers[ticker_symbol]
    
    # Fetch news for the ticker
    news_data = ticker.get_news()
    
    # Filter news if necessary and append to the all_news list
    for news in news_data:
        # Check if 'relatedTickers' exists and contains the current ticker symbol
        if 'relatedTickers' in news and ticker_symbol in news['relatedTickers']:
            # Add the ticker symbol to each news item for clarity
            news['ticker'] = ticker_symbol
            all_news.append(news)

# Sort all_news by 'providerPublishTime' in descending order to get the latest news first
all_news_sorted = sorted(all_news, key=lambda x: x['providerPublishTime'], reverse=True)

# Convert 'providerPublishTime' to a human-readable date and time format
for news_item in all_news_sorted:
    # Convert the timestamp to a datetime object
    if 'providerPublishTime' in news_item:
        news_item['providerPublishTime'] = datetime.utcfromtimestamp(news_item['providerPublishTime']).strftime('%Y-%m-%d %H:%M:%S')

# Create a DataFrame
all_news_df = pd.DataFrame(all_news_sorted, columns=['ticker', 'title', 'publisher', 'link', 'providerPublishTime'])

all_news_df


StatementMeta(, 8f0536b4-a6c3-4390-b53d-22459a26b32d, 51, Finished, Available)

,ticker,title,publisher,link,providerPublishTime
0,TSM,U.S. Seeks Semiconductor Supremacy with New Fu...,Oilprice.com,https://finance.yahoo.com/news/u-seeks-semicon...,2024-03-02 22:00:00
1,TSM,Japan lost its early lead in chips to companie...,Fortune,https://finance.yahoo.com/news/japan-lost-earl...,2024-03-02 21:00:00
2,NVDA,S&P adds two hot stocks to its flagship S&P 50...,TheStreet,https://finance.yahoo.com/m/04f08412-07d2-3dd8...,2024-03-02 20:21:43
3,NVDA,Is It Time To Ditch Tech Stocks for Gold?,Oilprice.com,https://finance.yahoo.com/news/time-ditch-tech...,2024-03-02 18:00:00
4,AMZN,"Rivian Automotive Stock Has 26% Upside, Accord...",Motley Fool,https://finance.yahoo.com/m/67f2aefd-af8c-35a0...,2024-03-02 17:04:00
...,...,...,...,...,...
115,LIN,Linde plc: Linde Reports Full-Year and Fourth-...,ACCESSWIRE,https://finance.yahoo.com/news/linde-plc-linde...,2024-02-06 11:10:00
116,LIN,Linde Signs Long-Term Agreements to Purchase R...,ACCESSWIRE,https://finance.yahoo.com/news/linde-signs-lon...,2024-02-01 11:20:00
117,LIN,Linde Starts up Supply of Clean Hydrogen and C...,ACCESSWIRE,https://finance.yahoo.com/news/linde-starts-su...,2024-01-30 11:30:00
118,LIN,Linde Expands Agreement with Steel Authority o...,ACCESSWIRE,https://finance.yahoo.com/news/linde-expands-a...,2024-01-25 11:20:00


In [38]:
# Convert the all_news_df pandas DataFrame to a Spark DataFrame
all_news_df = spark.createDataFrame(all_news_df)

# Show the Spark DataFrame to verify the structure
all_news_df.show()

StatementMeta(, 8f0536b4-a6c3-4390-b53d-22459a26b32d, 52, Finished, Available)

+------+--------------------+--------------------+--------------------+-------------------+
|ticker|               title|           publisher|                link|providerPublishTime|
+------+--------------------+--------------------+--------------------+-------------------+
|   TSM|U.S. Seeks Semico...|        Oilprice.com|https://finance.y...|2024-03-02 22:00:00|
|   TSM|Japan lost its ea...|             Fortune|https://finance.y...|2024-03-02 21:00:00|
|  NVDA|S&P adds two hot ...|           TheStreet|https://finance.y...|2024-03-02 20:21:43|
|  NVDA|Is It Time To Dit...|        Oilprice.com|https://finance.y...|2024-03-02 18:00:00|
|  AMZN|Rivian Automotive...|         Motley Fool|https://finance.y...|2024-03-02 17:04:00|
|  MSFT|Elon Musk vs. Eve...|The Wall Street J...|https://finance.y...|2024-03-02 16:51:00|
|  AAPL|Will Meta Platfor...|         Motley Fool|https://finance.y...|2024-03-02 16:30:00|
|  META|Will Meta Platfor...|         Motley Fool|https://finance.y...|2024-03-0

In [39]:
all_news_df.write.mode("overwrite").format("delta").save("Tables/" + 'NewInfo')

StatementMeta(, 8f0536b4-a6c3-4390-b53d-22459a26b32d, 53, Finished, Available)

In [40]:
import pandas as pd
import yfinance as yf
import openai
from openai import AzureOpenAI



client = AzureOpenAI(
    azure_endpoint="https://polite-ground-030dc3103.4.azurestaticapps.net/api/v1",
    api_key=openaiapi,
    api_version="2023-09-01-preview",
)



# Placeholder function for API call to an AI-based text summarization service
def generate_concise_summary(news_titles):
    messages = [
        {"role": "system", "content": "You are an AI trained to analyze financial news and predict their potential impact on stock prices. Provide concise, insightful summaries."},
        {"role": "user", "content": "Analyze the news titles below and summarize key points that could affect the stock's performance. Keep the summary under 200 characters and focus on potential market impacts."},
        {"role": "user", "content": news_titles}
    ]
    completion = client.chat.completions.create(model="gpt-4", messages=messages, temperature=0.7)
    summary = completion.choices[0].message.content if completion.choices else "Summary generation failed"
    
    
    return summary

# List of ticker symbols
ticker_symbols = 'MSFT AAPL 2222.SR GOOGL AMZN NVDA META BRK-A LLY TSM PEP SAP TMO LIN 601857.SS ROG.SW NVS MCD ABT DIS'

# Fetch data for all tickers
tickers = yf.Tickers(ticker_symbols)

data_list = []

# Iterate over tickers to fetch news and generate summaries
for ticker_symbol in ticker_symbols.split():
    ticker = tickers.tickers[ticker_symbol]
    
    # Fetch the latest news titles for the ticker
    news_data = ticker.get_news()
    news_titles = [news['title'] for news in news_data if 'title' in news]
    
    # Generate a concise summary for the collected news titles
    # For simplicity, join titles into one string; in practice, handle appropriately
    news_titles_str = ". ".join(news_titles)
    concise_summary = generate_concise_summary(news_titles_str)
    
    data_list.append({'Ticker': ticker_symbol, 'Concise Summary': concise_summary})

# Convert the list to a DataFrame
summary_df = pd.DataFrame(data_list)

summary_df


StatementMeta(, 8f0536b4-a6c3-4390-b53d-22459a26b32d, 54, Finished, Available)

,Ticker,Concise Summary
0,MSFT,1. Musk's AI developments could disrupt the se...
1,AAPL,1. Speculation on Meta's future growth could i...
2,2222.SR,"1. Meta Health's yearly loss has decreased, po..."
3,GOOGL,1. GoPro stock might be a good investment oppo...
4,AMZN,"1. Rivian stock may see a 26% rise, per Wall S..."
5,NVDA,"S&P 500 index adds two new stocks, potentially..."
6,META,1. Speculation around Meta's future valuation ...
7,BRK-A,1. Meta Health reports increased losses YoY in...
8,LLY,"The Nasdaq hit a record high, potentially boos..."
9,TSM,1. US semiconductor funding may boost related ...


In [41]:
# Convert the summary_df pandas DataFrame to a Spark DataFrame
summary_df = spark.createDataFrame(summary_df)

# Show the Spark DataFrame to verify the structure
summary_df.show()

StatementMeta(, 8f0536b4-a6c3-4390-b53d-22459a26b32d, 55, Finished, Available)

+---------+--------------------+
|   Ticker|     Concise Summary|
+---------+--------------------+
|     MSFT|1. Musk's AI deve...|
|     AAPL|1. Speculation on...|
|  2222.SR|1. Meta Health's ...|
|    GOOGL|1. GoPro stock mi...|
|     AMZN|1. Rivian stock m...|
|     NVDA|S&P 500 index add...|
|     META|1. Speculation ar...|
|    BRK-A|1. Meta Health re...|
|      LLY|The Nasdaq hit a ...|
|      TSM|1. US semiconduct...|
|      PEP|1. Pepsi's new gl...|
|      SAP|1. Stock market t...|
|      TMO|Thermo Fisher (TM...|
|      LIN|Linde's strong Q4...|
|601857.SS|1. Meta Health re...|
|   ROG.SW|1. Meta Health re...|
|      NVS|1. Novartis AG (N...|
|      MCD|1. Positive senti...|
|      ABT|Abbott (ABT) perf...|
|      DIS|1. TV networks fa...|
+---------+--------------------+



In [42]:
from pyspark.sql.functions import col

summary_df = summary_df.select([col(c).alias(
        c.replace( '(', '')
        .replace( ')', '')
        .replace( ',', '')
        .replace( ';', '')
        .replace( '{', '')
        .replace( '}', '')
        .replace( '\n', '')
        .replace( '\t', '')
        .replace( ' ', '_')
    ) for c in summary_df.columns])

StatementMeta(, 8f0536b4-a6c3-4390-b53d-22459a26b32d, 56, Finished, Available)

In [43]:
summary_df.write.mode("overwrite").format("delta").save("Tables/" + 'SummarizedNewInfo')

StatementMeta(, 8f0536b4-a6c3-4390-b53d-22459a26b32d, 57, Finished, Available)